<font color='red'>注：此处是文档第204页</font>

# 词嵌入：编码形式的词汇语义

## 2. Pytorch中的词嵌入
在我们举例或练习之前，这里有一份关于如何在`Pytorch`和常见的深度学习中使用词嵌入的简要介绍。与制作 `one-hot` 向量时对每个单词定义一个特殊的索引类似，当我们使用词向量时也需要为每个单词定义一个索引。这些索引将是查询表的关键点。意思就是，词嵌入被被存储在一个 $|V|\times D$ 的向量中，其中$D$是词嵌入的维度。词被被分配的索引`i`，表示在向量的第`i`行存储它的嵌入。 在所有的代码中，从单词到索引的映射是一个叫 `word_to_ix` 的字典。

能使用词嵌入的模块是 `torch.nn.Embedding` ，这里面有两个参数：**词汇表的大小**和**词嵌入的维度**。

索引这张表时，你必须使用 `torch.LongTensor` （因为索引是整数，不是浮点数）。

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [11]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor(list(word_to_ix.values()), dtype=torch.long)
print(lookup_tensor)
hello_embed = embeds(lookup_tensor)

print(hello_embed)

tensor([0, 1])
tensor([[ 0.8098,  0.0554,  1.1340, -0.5326,  0.6592],
        [-1.5964, -0.3769, -3.1020, -0.0995, -0.7213]],
       grad_fn=<EmbeddingBackward>)


## 3.例子： N-Gram语言模型
回想一下，在 `n-gram` 语言模型中,给定一个单词序列向量，我们要计算的是:
$$P(w_i|w_{i-1},w_{i-2},...,w_{i-n+1})$$
$w_i$是单词序列的第 i 个单词。 在本例中，我们将在训练样例上计算损失函数，并且用反向传播算法更新参数。

In [14]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# 我们用莎士比亚的十四行诗 Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# 应该对输入变量进行标记，但暂时忽略。
# 创建一系列的元组，每个元组都是([ word_i-2, word_i-1 ], target word)的形式。
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# 输出前3行，先看下是什么样子。
print(trigrams[:3])
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [15]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        # 步骤 1\. 准备好进入模型的数据 (例如将单词转换成整数索引,并将其封装在变量中)
        context_idxs = torch.tensor([word_to_ix[w] for w in context],
        dtype=torch.long)
        # 步骤 2\. 回调torch累乘梯度
        # 在传入一个新实例之前，需要把旧实例的梯度置零。
        model.zero_grad()
        # 步骤 3\. 继续运行代码，得到单词的log概率值。
        log_probs = model(context_idxs)
        # 步骤 4\. 计算损失函数（再次注意，Torch需要将目标单词封装在变量里）。
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], 
                                                  dtype=torch.long))
        # 步骤 5\. 反向传播更新梯度
        loss.backward()
        optimizer.step()
        # 通过调tensor.item()得到单个Python数值。
        total_loss += loss.item()
    losses.append(total_loss)

print(losses) # 用训练数据每次迭代，损失函数都会下降。

[522.686779499054, 520.2873945236206, 517.9037795066833, 515.5353033542633, 513.1816391944885, 510.84142565727234, 508.5146424770355, 506.19889974594116, 503.8932168483734, 501.595899105072]


## 4.练习：计算连续词袋模型的词向量
连续词袋模型（`CBOW`）在NLP深度学习中使用很频繁。它是一个模型，尝试通过目标词前后几个单词的文本，来预测目标词。这有别于语言模型， 因为`CBOW`不是序列的，也不必是概率性的。

`CBOW`常用于快速地训练词向量，得到的嵌入用来初始化一些复杂模型的嵌入。通常情况下，这被称为**预训练嵌入**。它几乎总能帮忙把模型性能提升几个百分点。

`CBOW` 模型如下所示：给定一个单词 $w_i$ ，$N$ 代表两边的滑窗距，如 $w_{i-1},...,w_{i-N}$ 和 $w_{i+1},...,w_{i+N}$ ，并将所有的上下文词统称为 $C$，`CBOW` 试图最小化

$$-logp(w_i|C)=-logSoftmax(A( \sum_{w\in C}q_w)+b)$$

其中 $q_w$ 是单词 $w_i$ 的嵌入。

在 Pytorch 中，通过填充下面的类来实现这个模型，有两条需要注意：
- 考虑下你需要定义哪些参数。
- 确保你知道每步操作后的结构，如果想重构，请使用 `.view()` 。

In [18]:
CONTEXT_SIZE = 2 # 左右各两个词
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()
# 通过对`raw_text`使用set()函数，我们进行去重操作
vocab = set(raw_text)
vocab_size = len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = []
    for j in range(1, CONTEXT_SIZE+1)[::-1]:
        context.append(raw_text[i - j])
    for j in range(1, CONTEXT_SIZE+1):
        context.append(raw_text[i + j])
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [25]:
class CBOW(nn.Module):
    def __init__(self):
        pass
    def forward(self, inputs):
        pass

# 创建模型并且训练。这里有些函数帮你在使用模块之前制作数据。
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix) # example

tensor([39, 41, 12, 35])